In [6]:
from matplotlib import pyplot as plt
%matplotlib inline
import h5py
import seaborn
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import pairwise_distances
import sys
sys.path.append("../")
from utils import l2_normalize, prewhiten, read_sampled_identities
from PIL import Image
seaborn.set()

In [32]:
model = tf.keras.models.load_model('../models/resnet50vggface2.h5')
model = tf.keras.Model(inputs=model.input, outputs=model.get_layer("dim_proj").output)

In [38]:
mean = np.float32(np.array([91.4953, 103.8827, 131.0912]))
base = "/data/vggface/test_preprocessed_sampled"
identities = os.listdir(base)

id2vect = {x: [] for x in identities}

for identity in identities:
    with h5py.File(os.path.join(base, identity, "images.h5"), 'r') as f:
        images = np.float32(f['images'][:])
        images -= mean
        id2vect[identity] = l2_normalize(model.predict(images))

ValueError: Error when checking input: expected base_input to have shape (224, 224, 3) but got array with shape (160, 160, 3)